In [45]:
import numpy as np
import re

In [2]:
padrino = [line.strip() for line in open("../documents/padrino.txt", encoding = "utf-8").readlines()]
padrino

['TÍTULO ORIGINAL',
 'The Godfather',
 '',
 'AÑO',
 '1972',
 '',
 'DURACIÓN',
 '175 min.',
 '',
 'PAÍS',
 'Estados Unidos',
 '',
 'DIRECCIÓN',
 'Francis Ford Coppola',
 '',
 'GUIÓN',
 'Francis Ford Coppola, Mario Puzo (Novela: Mario Puzo)',
 '',
 'MÚSICA',
 'Nino Rota',
 '',
 'FOTOGRAFÍA',
 'Gordon Willis',
 '',
 'REPARTO',
 'Marlon Brando, Al Pacino, James Caan, Robert Duvall, Diane Keaton, John Cazale, Talia Shire, Richard S. Castellano, Sterling Hayden, Gianni Russo, Rudy Bond, John Marley, Richard Conte, Al Lettieri, Abe Vigoda, Franco Citti, Lenny Montana, Al Martino, Joe Spinell, Simonetta Stefanelli, Morgana King, Alex Rocco, John Martino, Salvatore Corsitto, Richard Bright, Tony Giorgio, Vito Scotti, Jeannie Linero, Julie Gregg, Angelo Infanti, Corrado Gaipa, Saro Urzi',
 '',
 'PRODUCTORA',
 'Paramount Pictures, Alfran Productions',
 '',
 'GÉNERO',
 'Drama. Mafia. Crimen. Años 40. Años 50. Familia. Película de culto',
 '',
 'GRUPOS',
 'Trilogía El Padrino | Adaptaciones de Mari

In [154]:
class Data():

    def __init__(self, name):
        self.key = name

    def get_key(self):
        return self.key

    def _set_value(self, value):
        self.value = value

    def get_dic(self):
        return {self.key.lower(): self.value}

    def get_value(self, document):
        index_key = document.index(self.key)
        self._set_value(document[index_key + 1])
        return self


class ValoracionData(Data):

    def get_dic(self):
        return {"valoracion": {self.key[0].lower(): self.value[0], self.key[1].lower(): self.value[1]}}

    def get_value(self, document):
        index_key_puntuacion = document.index(self.key[0])
        index_key_votos = document.index(self.key[1])
        self._set_value(
            (document[index_key_puntuacion + 1], document[index_key_votos + 1]))
        return self


class PremiosData(Data):

    def get_dic(self):
        return {self.key[0].lower(): self.value}

    def get_value(self, document):
        index_key = document.index(self.key[0])
        next_index_key = document.index(self.key[1])
        values = document[index_key + 1:next_index_key-1]
        values = [self._preprocess_premio(value) for value in values]
        self._set_value(values)
        return self

    def _preprocess_premio(self, premio):
        values = premio.split(":")
        año = values[0].split(" ")[1]
        institucion = values[1].strip()
        categorias = values[2]
        return {"año": año, "institucion": institucion, "categorias": categorias}


class CriticasData(Data):

    def get_value(self, document):
        index_key = document.index(self.key)
        values = document[index_key + 1:]
        values = [self._preprocess_critica(value) for value in values]
        self._set_value(values)
        return self

    def _preprocess_critica(self, critica):
        match = re.search("(\".+\") (.+)", critica)
        critica = match.group(1).replace("\"", "")
        autor = match.group(2)
        return {"critica": critica, "autor": autor}


class PreprocessorDocument():

    HEADING_TITULO = "TÍTULO ORIGINAL"
    HEADING_AÑO = "AÑO"
    HEADING_DURACION = "DURACIÓN"
    HEADING_PAIS = "PAÍS"
    HEADING_DIRECCION = "DIRECCIÓN"
    HEADING_GUION = "GUIÓN"
    HEADING_MUSICA = "MÚSICA"
    HEADING_FOTOGRAFIA = "FOTOGRAFÍA"
    HEADING_REPARTO = "REPARTO"
    HEADING_PRODUCTORA = "PRODUCTORA"
    HEADING_GENERO = "GÉNERO"
    HEADING_GRUPOS = "GRUPOS"
    HEADING_SINOPSIS = "SINOPSIS"
    HEADING_PUNTUACION = "PUNTUACIÓN"
    HEADING_VOTOS = "VOTOS"
    HEADING_PREMIOS = "PREMIOS"
    HEADING_CRITICAS = "CRÍTICAS"

    def __init__(self, document):
        lines = [line.strip() for line in open(
            document, encoding="utf-8").readlines()]
        self.document = lines

    def preprocess(self):
        info = {}
        simple = [HEADING_TITULO, HEADING_AÑO, HEADING_DURACION, HEADING_PAIS, "DIRECCIÓN", HEADING_GUION, HEADING_MUSICA,
                  HEADING_FOTOGRAFIA, HEADING_REPARTO, HEADING_PRODUCTORA, HEADING_GENERO, HEADING_GRUPOS, HEADING_SINOPSIS]
        for heading in simple:
            info = dict(
                info, **Data(heading).get_value(self.document).get_dic())
        info = dict(info, **ValoracionData([HEADING_PUNTUACION,
                                            HEADING_VOTOS]).get_value(self.document).get_dic())
        info = dict(
            info, **PremiosData([HEADING_PREMIOS, HEADING_CRITICAS]).get_value(self.document).get_dic())
        info = dict(
            info, **CriticasData(HEADING_CRITICAS).get_value(self.document).get_dic())
        return info

In [155]:
info = PreprocessorDocument("../documents/psycho.txt").preprocess()
info

{'título original': 'Psycho',
 'año': '1960',
 'duración': '109 min.',
 'país': 'Estados Unidos Estados Unidos',
 'dirección': 'Alfred Hitchcock',
 'guión': 'Joseph Stefano (Novela: Robert Bloch)',
 'música': 'Bernard Herrmann',
 'fotografía': 'John L. Russell (B&W)',
 'reparto': 'Anthony Perkins, Janet Leigh, John Gavin, Vera Miles, John McIntire, Martin Balsam, Simon Oakland, Patricia Hitchcock',
 'productora': 'Paramount Pictures',
 'género': 'Terror. Intriga. Thriller. Película de culto. Asesinos en serie. Slasher. Thriller psicológico',
 'grupos': 'Psicosis',
 'sinopsis': 'Marion Crane, una joven secretaria, tras cometer el robo de un dinero en su empresa, huye de la ciudad y, después de conducir durante horas, decide descansar en un pequeño y apartado motel de carretera regentado por un tímido joven llamado Norman Bates, que vive en la casa de al lado con su madre. (FILMAFFINITY)',
 'valoracion': {'puntuación': '8,4', 'votos': '105.623'},
 'premios': [{'año': '1960',
   'instituc

In [151]:
data = Data(HEADING_TITULO)
data

In [106]:
data.get_value(padrino).get_dic()

{'título original': 'The Godfather'}

In [107]:
data.get_dic()

{'título original': 'The Godfather'}

In [108]:
valoracion = ValoracionData([HEADING_PUNTUACION, HEADING_VOTOS])

In [109]:
valoracion.get_value(padrino).get_dic()

{'valoracion': {'puntuación': '9,0', 'votos': '170.129'}}

In [110]:
a = {}

In [111]:
b = {"a": 2}

In [112]:
a = dict(a,**b)
a

{'a': 2}

In [113]:
PremiosData([HEADING_PREMIOS, HEADING_CRITICAS]).get_value(padrino).get_dic()

{'premios': [{'año': '1972',
   'institucion': '3 Oscars',
   'categorias': ' Mejor película, Actor (Marlon Brando), Guión adaptado. 11[10*] nom.'},
  {'año': '1972',
   'institucion': '5 Globos de Oro',
   'categorias': ' Película (Drama), Director, Actor (Brando), Guión y BSO'},
  {'año': '1972',
   'institucion': 'Premios BAFTA',
   'categorias': ' Mejor música. 5 nominaciones, incluyendo Mejor actor (Brando)'},
  {'año': '1972',
   'institucion': 'Círculo de Críticos de Nueva York',
   'categorias': ' Mejor actor secundario (Duvall). 4 nominaciones'},
  {'año': '1972',
   'institucion': 'National Board of Review',
   'categorias': ' Mejor actor sec. (Pacino) y Mejores 10 films del año'},
  {'año': '1972',
   'institucion': 'Sindicato de Directores (DGA)',
   'categorias': ' Mejor director'},
  {'año': '1972',
   'institucion': 'Sindicato de Guionistas (WGA)',
   'categorias': ' Mejor guión adaptado drama'},
  {'año': '1972',
   'institucion': 'Premios David di Donatello',
   'categ

In [114]:
critica = '- "Una gran película americana, llena de imágenes increíbles y momentos perdurables." Desson Thomson: The Washington Post'
"\"" in critica

True

In [115]:
import re

In [116]:
critica = re.search("(\".+\") (.+)", critica)

In [117]:
critica

<re.Match object; span=(2, 121), match='"Una gran película americana, llena de imágenes i>

In [118]:
z.group

<function Match.group>

In [121]:
a = CriticasData(HEADING_CRITICAS).get_value(padrino).get_dic()

In [122]:
a

{'críticas': [{'critica': "'El padrino' son palabras mayores. Las dos primeras partes están entre las 10 mejores películas de la historia del cine.",
   'autor': 'Carlos Boyero: Diario El Mundo'},
  {'critica': 'Coppola inventa una nueva mirada para el cine y amplía los horizontes de una industria que pedía a gritos savia nueva.',
   'autor': 'Luis Martínez: Diario El País'},
  {'critica': 'I believe in America... Así comienza este fascinante compendio de cine de Coppola. La compleja historia de una familia mafiosa liderada por un implacable padre de familia y hombre de honor (inmenso Marlon Brando) y cuyo poder hereda su hijo más pacífico (asombroso un primerizo Al Pacino de mirada gélida) es, ante todo, un ejercicio narrativo apabullante, de insuperable nivel. La brillante disección de todos los personajes, el ritmo magistral -que alterna largas secuencias familiares con creíbles escenas de acción- y una ambientación perfecta consiguen un film que entró violentamente entre los mejore